## 1. Setup & Configuration


In [1]:
# 1.1 Imports
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    StratifiedKFold,
    cross_validate,
    train_test_split,
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix,
    make_scorer,
)
from sklearn.base import clone
import lightgbm as lgb
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Imports réussis")

# 1.1.1 Paramètres globaux
RANDOM_STATE = 42
N_FOLDS = 5
COST_FN = 10  # Coût d'un Faux Négatif = 10x Faux Positif

# 1.2 Configuration MLFlow - CORRIGÉE
EXPERIMENT_NAME = "home_credit_default_risk"

# IMPORTANT : Vérifier si l'expérience existe, sinon la créer
try:
    # Essayer de récupérer l'expérience existante
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

    if experiment is None:
        # L'expérience n'existe pas, on la crée
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"✅ Expérience créée: {EXPERIMENT_NAME} (ID: {experiment_id})")
    else:
        # L'expérience existe déjà
        experiment_id = experiment.experiment_id
        print(f"✅ Expérience trouvée: {EXPERIMENT_NAME} (ID: {experiment_id})")

    # Définir l'expérience active
    mlflow.set_experiment(EXPERIMENT_NAME)

except Exception as e:
    print(f"⚠️ Erreur MLFlow: {e}")
    print("💡 Solution : Création manuelle de l'expérience...")
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
    mlflow.set_experiment(EXPERIMENT_NAME)
    print(f"✅ Expérience créée: {EXPERIMENT_NAME} (ID: {experiment_id})")

print(f"✅ Tracking URI: {mlflow.get_tracking_uri()}")
print(f"✅ Expérience active: {mlflow.get_experiment_by_name(EXPERIMENT_NAME).name}")

# 1.3 Charger les données preprocessed
DATA_PATH = Path("../data")
train_full = pd.read_csv(DATA_PATH / "train_preprocessed.csv")

print(f"✅ Données chargées: {train_full.shape}")

# Séparer X, y
X_full = train_full.drop(["TARGET", "SK_ID_CURR"], axis=1)
y_full = train_full["TARGET"]

# 🔀 SPLIT 80/20 pour créer un set de validation
X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full, test_size=0.2, stratify=y_full, random_state=RANDOM_STATE
)

print(f"✅ Train: {X_train.shape} - {y_train.value_counts(normalize=True).to_dict()}")
print(f"✅ Val:   {X_val.shape} - {y_val.value_counts(normalize=True).to_dict()}")
print(f"🔒 Val isolé jusqu'à l'évaluation finale")


print(f"✅ X_train shape: {X_train.shape}")
print(f"✅ y_train distribution: {y_train.value_counts().to_dict()}")


print("\n" + "=" * 60)
print("🎯 CONFIGURATION TERMINÉE")
print("=" * 60)

✅ Imports réussis


2026/02/13 16:12:36 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/13 16:12:36 INFO mlflow.store.db.utils: Updating database tables
2026/02/13 16:12:36 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/13 16:12:36 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/13 16:12:36 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/13 16:12:36 INFO alembic.runtime.migration: Will assume non-transactional DDL.


✅ Expérience trouvée: home_credit_default_risk (ID: 1)
✅ Tracking URI: sqlite:///mlflow.db
✅ Expérience active: home_credit_default_risk
✅ Données chargées: (307511, 421)
✅ Train: (246008, 419) - {0: 0.9192709180189262, 1: 0.08072908198107379}
✅ Val:   (61503, 419) - {0: 0.9192722306228964, 1: 0.08072776937710356}
🔒 Val isolé jusqu'à l'évaluation finale
✅ X_train shape: (246008, 419)
✅ y_train distribution: {0: 226148, 1: 19860}

🎯 CONFIGURATION TERMINÉE


## # ANALYSE DES MONTANTS


In [2]:
train_agregatted = pd.read_csv(DATA_PATH / "train_aggregated.csv")

# Montant moyen du crédit
amt_credit_mean = train_agregatted["AMT_CREDIT"].mean()
amt_credit_median = train_agregatted["AMT_CREDIT"].median()

print(f"📊 MONTANTS DE CRÉDIT:")
print(f"   Moyenne : {amt_credit_mean:,.0f}")
print(f"   Médiane : {amt_credit_median:,.0f}")
print(f"   Min     : {train_agregatted['AMT_CREDIT'].min():,.0f}")
print(f"   Max     : {train_agregatted['AMT_CREDIT'].max():,.0f}")

# Montant total des crédits
total_credits = train_agregatted["AMT_CREDIT"].sum()
print(f"\n💰 Total des crédits : {total_credits:,.0f}")

# Volume
n_clients = len(train_agregatted)
print(f"👥 Nombre de clients : {n_clients:,}")

# Taux de défaut dans les données
default_rate = train_agregatted["TARGET"].mean()
print(f"📉 Taux de défaut : {default_rate:.2%}")

📊 MONTANTS DE CRÉDIT:
   Moyenne : 599,026
   Médiane : 513,531
   Min     : 45,000
   Max     : 4,050,000

💰 Total des crédits : 184,207,084,196
👥 Nombre de clients : 307,511
📉 Taux de défaut : 8.07%


## 3. Score Métier Personnalisé

Le déséquilibre du coût métier :

- Faux Négatif (FN) : Mauvais client prédit bon → Crédit accordé → PERTE
- Faux Positif (FP) : Bon client prédit mauvais → Crédit refusé → Manque à gagner

Hypothèse : Coût FN = 10 × Coût FP


In [3]:
def business_cost_score(y_true, y_pred, cost_fn=10, cost_fp=1):
    """
    Calcule le coût métier total.

    Plus le score est BAS, meilleur est le modèle.

    Parameters:
    -----------
    y_true : array-like
        Vraies valeurs (0 ou 1)
    y_pred : array-like
        Prédictions (0 ou 1)
    cost_fn : float
        Coût d'un Faux Négatif
    cost_fp : float
        Coût d'un Faux Positif

    Returns:
    --------
    float : Coût métier total
    """
    from sklearn.metrics import confusion_matrix

    # Matrice de confusion
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Coût total
    total_cost = (fn * cost_fn) + (fp * cost_fp)

    # Normaliser par le nombre d'exemples
    cost_per_client = total_cost / len(y_true)

    return cost_per_client


def business_score_from_proba(y_true, y_proba, threshold=0.5, cost_fn=10, cost_fp=1):
    """
    Calcule le score métier à partir de probabilités et d'un seuil.
    """
    y_pred = (y_proba >= threshold).astype(int)
    return business_cost_score(y_true, y_pred, cost_fn, cost_fp)


# Test du score
print("\n📊 Test du Score Métier:")
print("=" * 60)

# Scénario 1 : Modèle parfait
y_true_test = np.array([0, 0, 1, 1])
y_pred_perfect = np.array([0, 0, 1, 1])
cost_perfect = business_cost_score(y_true_test, y_pred_perfect, COST_FN, 1)
print(f"Modèle parfait : Coût = {cost_perfect:.4f}")

# Scénario 2 : 1 Faux Négatif
y_pred_fn = np.array([0, 0, 0, 1])  # Manque 1 défaut
cost_fn = business_cost_score(y_true_test, y_pred_fn, COST_FN, 1)
print(f"1 Faux Négatif : Coût = {cost_fn:.4f} (= {COST_FN}/4 = {COST_FN / 4:.2f})")

# Scénario 3 : 1 Faux Positif
y_pred_fp = np.array([1, 0, 1, 1])  # Rejette 1 bon client
cost_fp = business_cost_score(y_true_test, y_pred_fp, COST_FN, 1)
print(f"1 Faux Positif : Coût = {cost_fp:.4f} (= 1/4 = {1 / 4:.2f})")

print(f"\n💡 Un FN coûte {cost_fn / cost_fp:.0f}x plus qu'un FP")


📊 Test du Score Métier:
Modèle parfait : Coût = 0.0000
1 Faux Négatif : Coût = 2.5000 (= 10/4 = 2.50)
1 Faux Positif : Coût = 0.2500 (= 1/4 = 0.25)

💡 Un FN coûte 10x plus qu'un FP


### Génère un rapport business complet à partir du score métier CV.

| Avantage         | Détail                                           |
| ---------------- | ------------------------------------------------ |
| **Réutilisable** | Un seul appel pour générer tout le rapport       |
| **Flexible**     | Paramètres personnalisables (LGD, marge, volume) |
| **Automatique**  | Charge les données et calcule tout seul          |
| **Comparable**   | Fonction bonus pour comparer plusieurs modèles   |
| **Intégrable**   | Retourne un dict de KPIs pour MLFlow             |
| **Sauvegardé**   | Génère un fichier .txt horodaté                  |

---


In [4]:
# ============================================
# GÉNÉRATEUR DE RAPPORT BUSINESS AUTOMATIQUE
# ============================================

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime


def generer_rapport_business(
    cv_business_cost,
    amt_credit_mean=None,
    taux_defaut=None,
    # Paramètres optionnels avec valeurs par défaut
    lgd=0.75,
    profit_margin=0.05,
    volume_annuel=100_000,
    projection_annees=3,
    # Chemins
    train_path="../data/train_aggregated.csv",
    output_dir="../reports",
    # Options
    sauvegarder=True,
    afficher=True,
):
    """
    Génère un rapport business complet à partir du score métier CV.

    Parameters:
    -----------
    cv_business_cost : float
        Score métier en cross-validation (ex: 0.4964)
    amt_credit_mean : float, optional
        Montant moyen du crédit. Si None, calculé depuis train_path
    taux_defaut : float, optional
        Taux de défaut. Si None, calculé depuis train_path
    lgd : float, default=0.75
        Loss Given Default (75% = standard crédit consommation)
    profit_margin : float, default=0.05
        Marge sur un bon client (5%)
    volume_annuel : int, default=100000
        Nombre de clients projetés par an
    projection_annees : int, default=3
        Nombre d'années pour la projection
    train_path : str
        Chemin vers train_aggregated.csv
    output_dir : str
        Répertoire de sortie pour le rapport
    sauvegarder : bool, default=True
        Sauvegarder le rapport en fichier txt
    afficher : bool, default=True
        Afficher le rapport dans le terminal

    Returns:
    --------
    dict : Dictionnaire avec tous les KPIs calculés
    """

    # ============================================
    # 1. CHARGER LES DONNÉES SI NÉCESSAIRE
    # ============================================

    if amt_credit_mean is None or taux_defaut is None:
        print(f"📂 Chargement de {train_path}...")
        try:
            train = pd.read_csv(train_path)

            if amt_credit_mean is None:
                amt_credit_mean = train["AMT_CREDIT"].mean()

            if taux_defaut is None:
                if "TARGET" not in train.columns:
                    raise ValueError("❌ Colonne TARGET absente. Vérifiez le fichier.")
                taux_defaut = train["TARGET"].mean()

            n_clients_historique = len(train)
            print(f"✅ Données chargées : {n_clients_historique:,} clients\n")

        except FileNotFoundError:
            raise FileNotFoundError(
                f"❌ Fichier non trouvé : {train_path}\n"
                "Veuillez fournir amt_credit_mean et taux_defaut manuellement."
            )
    else:
        n_clients_historique = None

    # ============================================
    # 2. CALCULS DES COÛTS UNITAIRES
    # ============================================

    cost_fn_euro = amt_credit_mean * lgd
    cost_fp_euro = amt_credit_mean * profit_margin
    ratio = cost_fn_euro / cost_fp_euro

    # ============================================
    # 3. CONVERSION DU SCORE EN EUROS
    # ============================================

    # Le score est normalisé avec cost_fp = 1
    # Donc : multiplier par cost_fp_euro pour avoir en €
    cout_modele_par_client = cv_business_cost * cost_fp_euro

    # ============================================
    # 4. BASELINE (MODÈLE NAÏF)
    # ============================================

    cout_naif_par_client = taux_defaut * cost_fn_euro

    # ============================================
    # 5. ÉCONOMIES
    # ============================================

    economie_par_client = cout_naif_par_client - cout_modele_par_client
    amelioration_pct = (economie_par_client / cout_naif_par_client) * 100

    # Impact annuel
    economie_annuelle = economie_par_client * volume_annuel

    # Projection multi-années
    economie_projection = economie_annuelle * projection_annees

    # ============================================
    # 6. ESTIMATION RÉPARTITION FN/FP
    # ============================================

    # Sur 1000 clients
    n_bons_1000 = int(1000 * (1 - taux_defaut))
    n_mauvais_1000 = int(1000 * taux_defaut)

    # Estimation inverse du nombre d'erreurs
    # cv_business_cost × 1000 = (fn × 10) + (fp × 1)
    total_cost_1000 = cv_business_cost * 1000

    # Estimation simplifiée (assume un équilibre raisonnable)
    # Ratio réel est 15:1, donc ajuster
    fn_estimate = int(total_cost_1000 / (10 + ratio / 15))
    fp_estimate = int((total_cost_1000 - fn_estimate * 10))

    # ============================================
    # 7. CONSTRUCTION DU RAPPORT
    # ============================================

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    rapport = f"""
╔════════════════════════════════════════════════════════════════════╗
║          RAPPORT D'IMPACT BUSINESS - SCORING CRÉDIT                ║
║          Généré le {timestamp}                       ║
╚════════════════════════════════════════════════════════════════════╝

┌────────────────────────────────────────────────────────────────────┐
│                          1. CONTEXTE                               │
└────────────────────────────────────────────────────────────────────┘

  Données de Base:
    • Montant moyen des crédits : {amt_credit_mean:,.0f} €
    • Taux de défaut historique : {taux_defaut:.2%}
"""

    if n_clients_historique:
        rapport += f"    • Échantillon analysé : {n_clients_historique:,} clients\n"

    rapport += f"""
  Hypothèses Métier:
    • Loss Given Default (LGD) : {lgd:.0%}
      → Perte moyenne en cas de défaut
    • Marge sur bon client : {profit_margin:.1%}
      → Profit moyen par client remboursant
    • Volume projeté : {volume_annuel:,} clients/an

┌────────────────────────────────────────────────────────────────────┐
│                   2. COÛTS DES ERREURS                             │
└────────────────────────────────────────────────────────────────────┘

  💥 Faux Négatif (FN) - Accepter un mauvais client:
    • Coût unitaire : {cost_fn_euro:,.0f} €
    • Calcul : {amt_credit_mean:,.0f} € × {lgd:.0%} = {cost_fn_euro:,.0f} €
    • Impact : Perte du capital prêté

  😕 Faux Positif (FP) - Refuser un bon client:
    • Coût unitaire : {cost_fp_euro:,.0f} €
    • Calcul : {amt_credit_mean:,.0f} € × {profit_margin:.1%} = {cost_fp_euro:,.0f} €
    • Impact : Manque à gagner (marge perdue)

  ⚖️  Ratio d'Importance:
    • Un FN coûte {ratio:.1f}× plus qu'un FP
    • Priorité : Détecter les mauvais clients

┌────────────────────────────────────────────────────────────────────┐
│                  3. PERFORMANCE DU MODÈLE                          │
└────────────────────────────────────────────────────────────────────┘

  📊 Score Métier (Cross-Validation):
    • Score normalisé : {cv_business_cost:.4f}
    • Coût réel par client : {cout_modele_par_client:,.0f} €

  📉 Baseline (Accepter tous les clients):
    • Coût par client : {cout_naif_par_client:,.0f} €
    • Calcul : {taux_defaut:.2%} × {cost_fn_euro:,.0f} € = {cout_naif_par_client:,.0f} €

  ✅ Amélioration:
    • Réduction du coût : {amelioration_pct:.1f}%
    • Économie par client : {economie_par_client:,.0f} €

  🔍 Estimation des Erreurs (sur 1000 clients):
    • {n_mauvais_1000} clients à risque → ~{fn_estimate} acceptés (FN)
    • {n_bons_1000} bons clients → ~{fp_estimate} refusés (FP)

┌────────────────────────────────────────────────────────────────────┐
│                    4. IMPACT FINANCIER                             │
└────────────────────────────────────────────────────────────────────┘

  💰 Par Client:
    ┌─────────────────────┬──────────────────┬──────────────────┐
    │                     │  Sans Modèle     │  Avec Modèle     │
    ├─────────────────────┼──────────────────┼──────────────────┤
    │ Coût par client     │ {cout_naif_par_client:>13,.0f} € │ {cout_modele_par_client:>13,.0f} € │
    │ Économie            │         -        │ {economie_par_client:>13,.0f} € │
    └─────────────────────┴──────────────────┴──────────────────┘

  📈 Impact Annuel ({volume_annuel:,} clients):
    • Coût sans modèle : {cout_naif_par_client * volume_annuel:,.0f} €
      = {(cout_naif_par_client * volume_annuel) / 1_000_000:.2f} millions €

    • Coût avec modèle : {cout_modele_par_client * volume_annuel:,.0f} €
      = {(cout_modele_par_client * volume_annuel) / 1_000_000:.2f} millions €

    • 💰 ÉCONOMIE ANNUELLE : {economie_annuelle:,.0f} €
                           = {economie_annuelle / 1_000_000:.2f} millions €

  🎯 Projection {projection_annees} ans:
"""

    for annee in range(1, projection_annees + 1):
        economie_cumul = economie_annuelle * annee
        rapport += f"    • Année {annee} (cumulé) : {economie_cumul / 1_000_000:.2f} millions €\n"

    rapport += f"""
    • 🏆 TOTAL {projection_annees} ANS : {economie_projection:,.0f} €
                     = {economie_projection / 1_000_000:.2f} millions €
                     = {economie_projection / 1_000_000_000:.2f} milliards €

┌────────────────────────────────────────────────────────────────────┐
│                      5. CONCLUSION                                 │
└────────────────────────────────────────────────────────────────────┘

  Le modèle de machine learning permet de réduire le coût par client
  de {amelioration_pct:.0f}%, en détectant mieux les clients à risque tout en
  maintenant un taux d'acceptation raisonnable des bons clients.

  ROI: Pour chaque {volume_annuel:,} clients traités par an, le modèle
       économise {economie_annuelle / 1_000_000:.2f} millions d'euros.

  Recommandation: ✅ Déployer le modèle en production

╚════════════════════════════════════════════════════════════════════╝
"""

    # ============================================
    # 8. AFFICHAGE
    # ============================================

    if afficher:
        print(rapport)

    # ============================================
    # 9. SAUVEGARDE
    # ============================================

    if sauvegarder:
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        filename = f"rapport_business_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        filepath = output_path / filename

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(rapport)

        print(f"\n✅ Rapport sauvegardé : {filepath}")

    # ============================================
    # 10. RETOUR DES KPIs
    # ============================================

    kpis = {
        # Entrées
        "cv_business_cost": cv_business_cost,
        "amt_credit_mean": amt_credit_mean,
        "taux_defaut": taux_defaut,
        "lgd": lgd,
        "profit_margin": profit_margin,
        "volume_annuel": volume_annuel,
        # Coûts unitaires
        "cost_fn_euro": cost_fn_euro,
        "cost_fp_euro": cost_fp_euro,
        "ratio_fn_fp": ratio,
        # Performance
        "cout_modele_par_client": cout_modele_par_client,
        "cout_naif_par_client": cout_naif_par_client,
        "economie_par_client": economie_par_client,
        "amelioration_pct": amelioration_pct,
        # Impact
        "economie_annuelle": economie_annuelle,
        "economie_projection": economie_projection,
        # Estimation erreurs
        "fn_estimate_per_1000": fn_estimate,
        "fp_estimate_per_1000": fp_estimate,
    }

    return kpis


## Fonction de train et de log des modeles pour selectionner le modele pour le grid search et la recherche d'hyperparametres


In [5]:
def train_and_log_model(model, model_name, X_train, y_train, params=None):
    """Entraîne un modèle et enregistre les métriques et artefacts dans MLflow.

    Args:
        model (_type_): Modèle à entraîner
        model_name (_type_): Nom du modèle
        X_train (_type_): Données d'entraînement
        y_train (_type_): Cibles d'entraînement
        params (_type_, optional): Paramètres du modèle. Defaults to None.

    Returns:
        _type_: Coût métier moyen sur la validation croisée
    """
    with mlflow.start_run(run_name=model_name):
        # --- 1. Paramètres & Métadonnées ---
        if params:
            mlflow.log_params(params)
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("train_shape", str(X_train.shape))
        mlflow.log_param(
            "target_dist", str(y_train.value_counts(normalize=True).to_dict())
        )

        # --- 2. Cross-Validation (Technique + Métier) ---
        scoring = {
            "auc": "roc_auc",
            "business": make_scorer(
                business_cost_score, cost_fn=COST_FN, cost_fp=1, greater_is_better=False
            ),
        }

        cv_results = cross_validate(
            model,
            X_train,
            y_train,
            cv=StratifiedKFold(
                n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE
            ),
            scoring=scoring,
            return_train_score=True,
            n_jobs=-1,
        )

        # --- 3. Entraînement final & Prédictions ---
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_train)[:, 1]

        # On utilise le seuil par défaut 0.5 pour la baseline
        y_pred_05 = (y_proba >= 0.5).astype(int)

        # --- 4. Calcul des métriques ---
        train_auc = roc_auc_score(y_train, y_proba)
        cv_auc_mean = cv_results["test_auc"].mean()

        # Matrice de confusion (pour extraire TN, FP, FN, TP)
        tn, fp, fn, tp = confusion_matrix(y_train, y_pred_05).ravel()

        # --- 5. LOGS MLFLOW  ---
        # CV Metrics
        mlflow.log_metric("cv_auc_mean", cv_auc_mean)
        mlflow.log_metric("cv_auc_std", cv_results["test_auc"].std())

        # ⚠️ IMPORTANT: greater_is_better=False fait que sklearn INVERSE le signe
        # cv_results["test_business"] contient des valeurs NÉGATIVES
        # On re-inverse avec `-` pour avoir un coût POSITIF dans MLFlow
        mlflow.log_metric("cv_business_cost_mean", -cv_results["test_business"].mean())

        # Overfitting Gaps
        mlflow.log_metric("auc_gap", train_auc - cv_auc_mean)

        # Confusion Matrix Metrics
        mlflow.log_metric("train_tn", int(tn))
        mlflow.log_metric("train_fp", int(fp))
        mlflow.log_metric("train_fn", int(fn))
        mlflow.log_metric("train_tp", int(tp))

        # --- 6. Artefacts (Plots & Modèle) ---
        fig, ax = plt.subplots(figsize=(6, 5))
        sns.heatmap(
            confusion_matrix(y_train, y_pred_05), annot=True, fmt="d", cmap="Blues"
        )
        mlflow.log_figure(fig, "confusion_matrix_default.png")
        plt.close()

        mlflow.sklearn.log_model(model, "model")

        print(
            f"✅ {model_name} | CV AUC: {cv_auc_mean:.4f} | Gap: {train_auc - cv_auc_mean:.4f}"
        )

        # Re-inverse pour retourner le coût positif (sklearn l'a inversé)
        return -cv_results["test_business"].mean()

In [7]:
# Définition des modèles à tester
models_to_test = [
    {
        "name": "Logistic_Regression_Baseline_balanced",
        "model": LogisticRegression(
            max_iter=1000, random_state=RANDOM_STATE, class_weight="balanced"
        ),
        "params": {"max_iter": 1000, "class_weight": "balanced"},
    },
    {
        "name": "Logistic_Regression_Baseline_non_balanced",
        "model": LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
        "params": {"max_iter": 1000},
    },
    {
        "name": "RandomForest_Baseline",
        "model": RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=RANDOM_STATE,
            class_weight="balanced",
            n_jobs=-1,
        ),
        "params": {"n_estimators": 100, "max_depth": 10},
    },
    {
        "name": "LightGBM_Baseline_balanced",
        "model": lgb.LGBMClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=7,
            random_state=RANDOM_STATE,
            class_weight="balanced",
        ),
        "params": {"n_estimators": 100, "learning_rate": 0.1},
    },
    {
        "name": "LightGBM_Baseline_non_balanced",
        "model": lgb.LGBMClassifier(
            n_estimators=100, learning_rate=0.1, max_depth=7, random_state=RANDOM_STATE
        ),
        "params": {"n_estimators": 100, "learning_rate": 0.1},
    },
]

# Exécution de la boucle
results_list = []

print("\n" + "=" * 60)
print("🚀 DÉMARRAGE DES ENTRAÎNEMENTS BASELINES")
print("=" * 60)

for m in models_to_test:
    score = train_and_log_model(
        m["model"], m["name"], X_train, y_train, params=m["params"]
    )
    results_list.append({"Model": m["name"], "CV Business Cost": score})

# 3. Comparaison finale
print("\n" + "=" * 60)
print("📊 RÉSULTATS FINAUX")
print("=" * 60)
baseline_results = pd.DataFrame(results_list).sort_values("CV Business Cost")
print(baseline_results.to_string(index=False))

best_baseline = baseline_results.iloc[0]["Model"]
print(f"\n💡 Le meilleur modèle est : {best_baseline}")


🚀 DÉMARRAGE DES ENTRAÎNEMENTS BASELINES


2026/02/13 16:14:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/13 16:14:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


✅ Logistic_Regression_Baseline_balanced | CV AUC: 0.7705 | Gap: 0.0057


2026/02/13 16:14:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/13 16:14:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


✅ Logistic_Regression_Baseline_non_balanced | CV AUC: 0.7700 | Gap: 0.0048


...

✅ RandomForest_Baseline | CV AUC: 0.7545 | Gap: 0.0788
[LightGBM] [Info] Number of positive: 15888, number of negative: 180918
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.145657 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 46867
[LightGBM] [Info] Number of data points in the train set: 196806, number of used features: 401
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 15888, number of negative: 180918
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.448147 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 46893
[LightGBM] [Info] Number of data points in the train

2026/02/13 16:16:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/13 16:16:38 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


✅ LightGBM_Baseline_balanced | CV AUC: 0.7785 | Gap: 0.0442
[LightGBM] [Info] Number of positive: 15888, number of negative: 180918
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.467164 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 46868
[LightGBM] [Info] Number of data points in the train set: 196806, number of used features: 402
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432480
[LightGBM] [Info] Start training from score -2.432480
[LightGBM] [Info] Number of positive: 15888, number of negative: 180919
[LightGBM] [Info] Number of positive: 15888, number of negative: 180918
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.309859 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGB

2026/02/13 16:17:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/13 16:17:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


✅ LightGBM_Baseline_non_balanced | CV AUC: 0.7782 | Gap: 0.0460

📊 RÉSULTATS FINAUX
                                    Model  CV Business Cost
               LightGBM_Baseline_balanced          0.499955
    Logistic_Regression_Baseline_balanced          0.511394
                    RandomForest_Baseline          0.535344
           LightGBM_Baseline_non_balanced          0.784300
Logistic_Regression_Baseline_non_balanced          0.787718

💡 Le meilleur modèle est : LightGBM_Baseline_balanced


In [8]:
"""
## 4. Hyperparameter Tuning avec GridSearchCV (Harmonisé)

On optimise le meilleur modèle baseline avec GridSearchCV.
On utilise le multi-scoring pour logger l'AUC et le Score Métier en même temps.
"""

best_baseline_name = "LightGBM_Baseline_balanced"

print(f"\n🔍 Hyperparameter Tuning sur {best_baseline_name}")
print("=" * 60)

# 1. Définition des scorers (AUC + Métier)
scoring = {
    "AUC": "roc_auc",
    "Business_Score": make_scorer(
        business_cost_score, greater_is_better=False, cost_fn=COST_FN, cost_fp=1
    ),
}

# 2. Grille de paramètres
param_grid = {
    "n_estimators": [100, 200],
    "learning_rate": [0.01, 0.05, 0.1],
    "num_leaves": [31, 50, 70],  # contrôle la complexité
    "min_child_samples": [20, 50, 100],  # régularisation pour le déséquilibre
    "force_row_wise": [True],
    "verbosity": [-1],
}

# LightGBM utilise des arbres leaf-wise. La règle est : num_leaves <= 2^max_depth. Or ta grille teste :

# max_depth=5 + num_leaves=70 → 70 > 2^5 = 32, donc num_leaves est ignoré silencieusement
# max_depth=5 + num_leaves=50 → idem
# Ça veut dire que tu gaspilles du temps de calcul sur des combinaisons qui reviennent au même. Deux options :

# Retirer max_depth et ne garder que num_leaves (approche recommandée LightGBM)
# Ou garder max_depth et ajuster num_leaves pour qu'il soit cohérent


lgbm_grid = lgb.LGBMClassifier(
    random_state=RANDOM_STATE, class_weight="balanced", n_jobs=-1
)

# 3. GridSearchCV
grid_search = GridSearchCV(
    lgbm_grid,  # type:ignore
    param_grid,
    cv=StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE),
    scoring=scoring,
    refit="Business_Score",  # Crucial: On choisit le meilleur modèle selon le coût métier
    return_train_score=True,  # Pour calculer les gaps d'overfitting
    n_jobs=-1,
    verbose=2,
)

print("⏳ GridSearch en cours...")
grid_search.fit(X_train, y_train)

# --- 4. LOG HARMONISÉ DANS MLFLOW ---

with mlflow.start_run(run_name=f"{best_baseline_name}_Tuned_Complete"):
    # Index du meilleur run
    idx = grid_search.best_index_
    best_model = grid_search.best_estimator_

    # A. Paramètres
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_param("model_type", f"{best_baseline_name}_Tuned")
    mlflow.log_param("n_features", X_train.shape[1])

    # B. Métriques de Cross-Validation (Moyennes et Écarts-types)
    cv_auc_mean = grid_search.cv_results_["mean_test_AUC"][idx]
    cv_cost_mean = -grid_search.cv_results_["mean_test_Business_Score"][idx]

    mlflow.log_metric("cv_auc_mean", cv_auc_mean)
    mlflow.log_metric("cv_auc_std", grid_search.cv_results_["std_test_AUC"][idx])
    mlflow.log_metric("cv_business_cost_mean", cv_cost_mean)
    mlflow.log_metric(
        "cv_business_cost_std", grid_search.cv_results_["std_test_Business_Score"][idx]
    )

    # C. Métriques sur le Train (pour détection Overfitting)
    y_proba = best_model.predict_proba(X_train)[:, 1]
    y_pred_05 = (y_proba >= 0.5).astype(int)  # Seuil baseline

    train_auc = roc_auc_score(y_train, y_proba)

    # D. Gaps & Confusion Matrix
    mlflow.log_metric("auc_gap", train_auc - cv_auc_mean)

    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_05).ravel()
    mlflow.log_metric("train_fn", int(fn))
    mlflow.log_metric("train_fp", int(fp))

    # E. Artefacts
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(confusion_matrix(y_train, y_pred_05), annot=True, fmt="d", cmap="Blues")
    ax.set_title(f"Confusion Matrix - {best_baseline_name} Tuned (Seuil 0.5)")
    mlflow.log_figure(fig, "confusion_matrix_tuned.png")
    plt.close()

    # F. Modèle
    mlflow.sklearn.log_model(best_model, "model")

    print(f"\n✅ Tuning terminé et loggué de façon exhaustive.")
    print(f"Meilleurs paramètres: {grid_search.best_params_}")
    print(f"Meilleur CV AUC: {cv_auc_mean:.4f} | CV Business Cost: {cv_cost_mean:.4f}")


🔍 Hyperparameter Tuning sur LightGBM_Baseline_balanced
⏳ GridSearch en cours...
Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END force_row_wise=True, learning_rate=0.01, min_child_samples=20, n_estimators=100, num_leaves=31, verbosity=-1; total time= 1.2min
[CV] END force_row_wise=True, learning_rate=0.01, min_child_samples=20, n_estimators=100, num_leaves=31, verbosity=-1; total time= 1.2min
[CV] END force_row_wise=True, learning_rate=0.01, min_child_samples=20, n_estimators=100, num_leaves=31, verbosity=-1; total time= 1.2min
[CV] END force_row_wise=True, learning_rate=0.01, min_child_samples=20, n_estimators=100, num_leaves=31, verbosity=-1; total time= 1.2min
[CV] END force_row_wise=True, learning_rate=0.01, min_child_samples=20, n_estimators=100, num_leaves=31, verbosity=-1; total time= 1.3min
[CV] END force_row_wise=True, learning_rate=0.01, min_child_samples=20, n_estimators=100, num_leaves=50, verbosity=-1; total time= 1.4min
[CV] END force_row_wise=True,

2026/02/13 17:04:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/13 17:04:46 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



✅ Tuning terminé et loggué de façon exhaustive.
Meilleurs paramètres: {'force_row_wise': True, 'learning_rate': 0.05, 'min_child_samples': 50, 'n_estimators': 200, 'num_leaves': 50, 'verbosity': -1}
Meilleur CV AUC: 0.7814 | CV Business Cost: 0.4951


In [9]:
def find_optimal_threshold(y_true, y_proba, cost_fn=10, cost_fp=1):
    """
    Trouve le seuil optimal.
    Note: On élargit l'intervalle car avec FN=10, le seuil est souvent bas.
    """
    thresholds = np.linspace(0.02, 0.7, 100)  # Plus de précision sur les seuils bas
    costs = []

    for thresh in thresholds:
        y_pred = (y_proba >= thresh).astype(int)
        cost = business_cost_score(y_true, y_pred, cost_fn, cost_fp)
        costs.append(cost)

    optimal_idx = np.argmin(costs)
    optimal_threshold = thresholds[optimal_idx]
    optimal_cost = costs[optimal_idx]

    # Figure d'optimisation
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(thresholds, costs, color="navy", lw=2)
    ax.axvline(
        optimal_threshold,
        color="red",
        linestyle="--",
        label=f"Optimal: {optimal_threshold:.3f}",
    )
    ax.axvline(0.5, color="orange", linestyle=":", label="Défaut: 0.500")
    ax.set_title("Recherche du Seuil Minimisant le Coût Métier")
    ax.set_xlabel("Seuil")
    ax.set_ylabel("Coût Métier")
    ax.legend()

    return optimal_threshold, optimal_cost, fig


def find_optimal_threshold_cv(
    model, X_train, y_train, cost_fn=10, cost_fp=1, n_folds=5
):
    """
    Version CROSS-VALIDATION - évite l'overfitting du seuil.
    Utilise Out-Of-Fold predictions.
    """
    from sklearn.base import clone

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    y_proba_oof = np.zeros(len(y_train))

    print(f"🔄 Optimisation du seuil en {n_folds}-fold CV...")

    # Générer les prédictions OOF
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        X_tr = X_train.iloc[train_idx]
        y_tr = y_train.iloc[train_idx]
        X_vl = X_train.iloc[val_idx]

        # Cloner et entraîner
        model_fold = clone(model)
        model_fold.fit(X_tr, y_tr)

        # Prédire sur validation fold
        y_proba_oof[val_idx] = model_fold.predict_proba(X_vl)[:, 1]

        print(f"  Fold {fold_idx + 1}/{n_folds} terminé")

    # Chercher le seuil optimal sur OOF
    thresholds = np.linspace(0.02, 0.7, 100)
    costs = []

    for thresh in thresholds:
        y_pred = (y_proba_oof >= thresh).astype(int)
        cost = business_cost_score(y_train, y_pred, cost_fn, cost_fp)
        costs.append(cost)

    optimal_idx = np.argmin(costs)
    optimal_threshold = thresholds[optimal_idx]
    optimal_cost = costs[optimal_idx]

    # Figure
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(thresholds, costs, color="navy", lw=2)
    ax.axvline(
        optimal_threshold,
        color="red",
        linestyle="--",
        label=f"Optimal (CV): {optimal_threshold:.3f}",
    )
    ax.axvline(0.5, color="orange", linestyle=":", label="Défaut: 0.500")
    ax.set_title("Optimisation du Seuil en Cross-Validation (Sans Fuite)")
    ax.set_xlabel("Seuil")
    ax.set_ylabel("Coût Métier")
    ax.legend()
    plt.close()

    return optimal_threshold, optimal_cost, fig, y_proba_oof


# --- Exécution ---

# ✅ CORRECTION : Génération des prédictions OOF pour éviter le data leakage
optimal_thresh, optimal_cost, fig_thresh, y_proba_oof = find_optimal_threshold_cv(
    best_model, X_train, y_train, COST_FN, 1, N_FOLDS
)

# ✅ CORRECTION : Utiliser y_proba_oof au lieu de y_train_proba
y_pred_opt = (y_proba_oof >= optimal_thresh).astype(int)
y_pred_def = (y_proba_oof >= 0.5).astype(int)

# --- LOG FINAL HARMONISÉ ---

with mlflow.start_run(run_name=f"{best_baseline}_FINAL_MODEL"):
    # 1. Hyperparamètres et Seuil
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_param("optimal_threshold", round(optimal_thresh, 3))
    mlflow.log_param("model_type", f"{best_baseline}_Tuned_Final")

    # 2. Métriques de Cross-Validation (Harmonisation avec les Baselines)
    # On récupère les résultats du GridSearch pour le meilleur index
    best_idx = grid_search.best_index_

    cv_auc = grid_search.cv_results_["mean_test_AUC"][best_idx]
    cv_cost = -grid_search.cv_results_["mean_test_Business_Score"][
        best_idx
    ]  # On repasse en positif
    cv_cost_std = grid_search.cv_results_["std_test_Business_Score"][best_idx]

    mlflow.log_metric("cv_auc_mean", cv_auc)
    mlflow.log_metric("cv_business_cost_mean", cv_cost)
    mlflow.log_metric("cv_business_cost_std", cv_cost_std)

    # 3. Métriques de performance sur le Train (Technique)
    # ✅ CORRECTION : Calcul de train_auc sur OOF pour cohérence
    train_auc = roc_auc_score(y_train, y_proba_oof)
    mlflow.log_metric("train_auc", train_auc)
    mlflow.log_metric("auc_gap", train_auc - cv_auc)

    # 4. Impact Métier du Seuil (La valeur ajoutée de cette étape)
    cost_def = business_cost_score(y_train, y_pred_def, COST_FN, 1)
    mlflow.log_metric("train_business_cost_at_05", cost_def)
    mlflow.log_metric("train_business_cost_optimal", optimal_cost)

    improvement = ((cost_def - optimal_cost) / cost_def) * 100
    mlflow.log_metric("threshold_improvement_percent", improvement)

    # 5. Confusion Matrix au seuil OPTIMAL
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_opt).ravel()
    mlflow.log_metric("final_train_fn", int(fn))
    mlflow.log_metric("final_train_fp", int(fp))
    mlflow.log_metric("final_train_tp", int(tp))
    mlflow.log_metric("final_train_tn", int(tn))

    # 6. Artefacts
    mlflow.log_figure(fig_thresh, "threshold_optimization_curve.png")

    fig_cm, ax = plt.subplots()
    sns.heatmap(
        confusion_matrix(y_train, y_pred_opt), annot=True, fmt="d", cmap="Greens"
    )
    ax.set_title(f"Matrice de Confusion au Seuil Optimal ({optimal_thresh:.3f})")
    mlflow.log_figure(fig_cm, "final_confusion_matrix.png")
    plt.close()

    # 7. Model Registry
    mlflow.sklearn.log_model(
        best_model, "model", registered_model_name="Credit_Scoring_Model_LGBM"
    )

    print(f"✅ Modèle Final loggué et enregistré !")
    print(f"Score CV métier: {cv_cost:.4f} (± {cv_cost_std:.4f})")
    print(
        f"Gain métier sur train (seuil 0.5 -> {optimal_thresh:.3f}): {improvement:.1f}%"
    )

🔄 Optimisation du seuil en 5-fold CV...
  Fold 1/5 terminé
  Fold 2/5 terminé
  Fold 3/5 terminé
  Fold 4/5 terminé
  Fold 5/5 terminé


2026/02/13 17:07:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/13 17:07:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/02/13 17:07:42 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/13 17:07:42 INFO mlflow.store.db.utils: Updating database tables
2026/02/13 17:07:42 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/13 17:07:42 INFO alembic.runtime.migration: Will assume non-transactional DDL.


✅ Modèle Final loggué et enregistré !
Score CV métier: 0.4951 (± 0.0068)
Gain métier sur train (seuil 0.5 -> 0.494): 0.2%


Successfully registered model 'Credit_Scoring_Model_LGBM'.
Created version '1' of model 'Credit_Scoring_Model_LGBM'.


In [ ]:
"""
## 5. Evaluation finale sur le set de validation (X_val)

Ce set a été isolé dès le début (20% des données) et n'a jamais été
utilisé pendant l'entraînement ni le tuning. C'est notre estimation
honnête de la performance en production.
"""

print("\n" + "=" * 60)
print("🎯 ÉVALUATION FINALE SUR LE SET DE VALIDATION (20%)")
print("=" * 60)

# Prédictions sur X_val
y_val_proba = best_model.predict_proba(X_val)[:, 1]
y_val_pred_optimal = (y_val_proba >= optimal_thresh).astype(int)
y_val_pred_default = (y_val_proba >= 0.5).astype(int)

# Métriques
val_auc = roc_auc_score(y_val, y_val_proba)
val_cost_optimal = business_cost_score(y_val, y_val_pred_optimal, COST_FN, 1)
val_cost_default = business_cost_score(y_val, y_val_pred_default, COST_FN, 1)

# Confusion matrix au seuil optimal
tn_val, fp_val, fn_val, tp_val = confusion_matrix(y_val, y_val_pred_optimal).ravel()

print(f"\n📊 AUC Validation : {val_auc:.4f}")
print(f"📊 Business Cost (seuil 0.5)     : {val_cost_default:.4f}")
print(f"📊 Business Cost (seuil {optimal_thresh:.3f}) : {val_cost_optimal:.4f}")
print(f"\n📊 Matrice de confusion (seuil {optimal_thresh:.3f}) :")
print(f"   TN={tn_val:,}  FP={fp_val:,}")
print(f"   FN={fn_val:,}  TP={tp_val:,}")

# Comparaison CV vs Validation
print(f"\n🔍 Comparaison CV vs Validation :")
print(f"   CV AUC          : {cv_auc:.4f}")
print(f"   Val AUC         : {val_auc:.4f}")
print(f"   Écart           : {abs(cv_auc - val_auc):.4f}")
print(f"   CV Business Cost: {cv_cost:.4f}")
print(f"   Val Business Cost: {val_cost_optimal:.4f}")

if abs(cv_auc - val_auc) < 0.02:
    print(
        "\n✅ Les scores CV et validation sont cohérents — le modèle généralise bien."
    )
else:
    print("\n⚠️ Écart significatif entre CV et validation — possible overfitting.")

# Log dans MLflow
with mlflow.start_run(run_name=f"{best_baseline}_VALIDATION"):
    mlflow.log_metric("val_auc", val_auc)
    mlflow.log_metric("val_business_cost_optimal", val_cost_optimal)
    mlflow.log_metric("val_business_cost_default", val_cost_default)
    mlflow.log_metric("val_fn", int(fn_val))
    mlflow.log_metric("val_fp", int(fp_val))
    mlflow.log_metric("val_tn", int(tn_val))
    mlflow.log_metric("val_tp", int(tp_val))
    mlflow.log_param("optimal_threshold", round(optimal_thresh, 3))
    mlflow.log_param("model_type", f"{best_baseline}_Validation")

    fig_val, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(
        confusion_matrix(y_val, y_val_pred_optimal), annot=True, fmt="d", cmap="Purples"
    )
    ax.set_title(f"Validation - Confusion Matrix (Seuil {optimal_thresh:.3f})")
    mlflow.log_figure(fig_val, "validation_confusion_matrix.png")
    plt.close()

print("\n✅ Évaluation validation loguée dans MLflow")


🎯 ÉVALUATION FINALE SUR LE SET DE VALIDATION (20%)

📊 AUC Validation : 0.7852
📊 Business Cost (seuil 0.5)     : 0.4908
📊 Business Cost (seuil 0.494) : 0.4907

📊 Matrice de confusion (seuil 0.494) :
   TN=41,659  FP=14,879
   FN=1,530  TP=3,435

🔍 Comparaison CV vs Validation :
   CV AUC          : 0.7814
   Val AUC         : 0.7852
   Écart           : 0.0038
   CV Business Cost: 0.4951
   Val Business Cost: 0.4907

✅ Les scores CV et validation sont cohérents — le modèle généralise bien.

✅ Évaluation validation loguée dans MLflow


In [12]:
"""
## 6. Comparaison Finale et Sélection du Meilleur Modèle
"""

print("\n" + "=" * 60)
print("📊 COMPARAISON FINALE DES MODÈLES")
print("=" * 60)

# Récupérer tous les runs de l'experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# Trier par business_cost
runs_sorted = runs.sort_values("metrics.cv_business_cost_mean", ascending=True)

# Afficher le top 10 des modèles
print("\nTop 10 Modèles (triés par coût métier):")
columns_to_show = [
    "tags.mlflow.runName",
    "metrics.cv_business_cost_mean",  # Correct
    "metrics.train_auc",  # Correct
    "metrics.cv_auc_mean",  # Ajout du préfixe 'metrics.' ici !
    "metrics.auc_gap",  # Si tu l'as loggué, c'est très utile à voir
]

# On ne garde que les colonnes qui existent réellement pour éviter un nouveau crash
existing_cols = [c for c in columns_to_show if c in runs_sorted.columns]

print(runs_sorted[existing_cols].head(10).to_string(index=False))
# Sélectionner le meilleur
best_run_id = runs_sorted.iloc[0]["run_id"]
best_run_name = runs_sorted.iloc[0]["tags.mlflow.runName"]

print(f"\n✅ Meilleur modèle: {best_run_name}")
print(f"✅ Run ID: {best_run_id}")


📊 COMPARAISON FINALE DES MODÈLES

Top 10 Modèles (triés par coût métier):
                      tags.mlflow.runName  metrics.cv_business_cost_mean  metrics.train_auc  metrics.cv_auc_mean  metrics.auc_gap
   LightGBM_Baseline_balanced_FINAL_MODEL                       0.495069           0.781337             0.781354        -0.000016
LightGBM_Baseline_balanced_Tuned_Complete                       0.495069                NaN             0.781354         0.070313
LightGBM_Baseline_balanced_Tuned_Complete                       0.498224                NaN             0.779327         0.044570
               LightGBM_Baseline_balanced                       0.499955                NaN             0.778513         0.044177
               LightGBM_Baseline_balanced                       0.499955                NaN             0.778513         0.044177
    Logistic_Regression_Baseline_balanced                       0.511394                NaN             0.770536         0.005719
    Logistic_Re

In [13]:
"""
## 7. Chargement des Données de Test
"""

print("\n" + "=" * 60)
print("📂 CHARGEMENT DES DONNÉES DE TEST")
print("=" * 60)

# Charger les données de test preprocessed
test_full = pd.read_csv(DATA_PATH / "test_preprocessed.csv")

# Séparer les IDs et les features
test_ids = test_full["SK_ID_CURR"]
X_test = test_full.drop("SK_ID_CURR", axis=1)

print(f"✅ Données de test chargées: {X_test.shape}")
print(f"✅ IDs de test: {len(test_ids)} clients")
print(f"✅ Prêt pour les prédictions!")


📂 CHARGEMENT DES DONNÉES DE TEST
✅ Données de test chargées: (48744, 420)
✅ IDs de test: 48744 clients
✅ Prêt pour les prédictions!


In [ ]:
"""
## 8. Model Serving - Test du Modèle en Production
"""
# S'assurer que le nom correspond à ce qui a été enregistré plus haut
model_name = "Credit_Scoring_Model_LGBM"

# Charger la dernière version (au lieu de "Production" qui est peut-être vide)
model_uri = f"models:/{model_name}/latest"
print(f"Chargement du modèle depuis : {model_uri}")

model_production = mlflow.sklearn.load_model(
    model_uri
)  # Utilise sklearn.load_model pour garder predict_proba

# 1. Aligner X_test sur les features du modèle de façon robuste
features_cols = model_production.feature_name_

# reindex ajoute les colonnes manquantes (fill_value=0) et retire les colonnes en trop
X_test_aligned = X_test.reindex(columns=features_cols, fill_value=0)

print(
    f"✅ Alignement terminé : {X_test.shape[1]} -> {X_test_aligned.shape[1]} colonnes"
)

# 2. Test sur l'échantillon
print(f"\n🧪 Test du Modèle en Production :")
print("=" * 60)

X_sample = X_test_aligned.head(5)
predictions_proba = model_production.predict_proba(X_sample)[:, 1]
predictions = (predictions_proba >= optimal_thresh).astype(int)

# 3. Affichage
result_df = pd.DataFrame(
    {
        "SK_ID_CURR": test_ids.head(5).values,
        "Probability": predictions_proba,
        "Prediction": predictions,
    }
)
print(result_df.to_string(index=False))

# 4. Génération du fichier final
print("\n📤 Génération des prédictions finales pour Kaggle...")
final_proba = model_production.predict_proba(X_test_aligned)[:, 1]

submission = pd.DataFrame(
    {
        "SK_ID_CURR": test_ids,
        "TARGET": final_proba,  # On envoie la proba pour maximiser l'AUC sur Kaggle
    }
)

submission.to_csv("../data/submission.csv", index=False)
print("✅ Fichier de soumission créé : ../data/submission.csv")

Chargement du modèle depuis : models:/Credit_Scoring_Model_LGBM/latest


✅ Alignement terminé : 420 -> 419 colonnes

🧪 Test du Modèle en Production :
 SK_ID_CURR  Probability  Prediction
     100001     0.242050           0
     100005     0.635779           1
     100013     0.248509           0
     100028     0.180654           0
     100038     0.518640           1

📤 Génération des prédictions finales pour Kaggle...
✅ Fichier de soumission créé : ../data/submission.csv


In [ ]:
"""
## 9. Documentation et Export
"""

# Sauvegarder le seuil optimal et autres métadonnées
metadata = {
    "best_model_name": best_run_name,
    "best_model_run_id": best_run_id,
    "optimal_threshold": optimal_thresh,
    "cost_fn": COST_FN,
    "cost_fp": 1,
    "business_cost_optimal": optimal_cost,  # ✅ CORRECTION : cost_optimal → optimal_cost
    "n_features": X_train.shape[1],
    "n_train_samples": X_train.shape[0],
    "class_distribution": y_train.value_counts().to_dict(),
}

import json

with open("../models/model_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print("\n✅ Métadonnées sauvegardées: model_metadata.json")

print("\n" + "=" * 60)
print("🎉 MODELING TERMINÉ !")
print("=" * 60)
print(f"""
RÉSUMÉ:
- Meilleur modèle: {best_run_name}
- Coût métier: {optimal_cost:.4f}
- Seuil optimal: {optimal_thresh:.3f}
- Modèle en production: {model_name}
- Fichier soumission: submission.csv

PROCHAINES ÉTAPES:
1. Lancer l'UI MLFlow: mlflow ui
2. Comparer les runs et visualisations
3. Soumettre à Kaggle
4. Documenter les résultats
""")